In [1]:
import pandas as pd
import numpy as np

import re
from itertools import chain
from collections import Counter

from google.cloud import bigquery
from google.oauth2 import service_account

from datetime import datetime, timedelta

import requests
import warnings
from requests.packages.urllib3.exceptions import InsecureRequestWarning

# InsecureRequestWarning 경고 무시
warnings.simplefilter('ignore', InsecureRequestWarning)

In [2]:
KEY_PATH = ".config/"
servicekey_path = KEY_PATH + "serviceKey.json" ## 빅쿼리 외 다른 API 활용 위해
bigquerykey_path = KEY_PATH + "mido-project-426906-31b49963ac97.json"

warnings.filterwarnings("ignore")

In [3]:
# BigQuery 클라이언트 생성 함수
def create_bigquery_client(key_path):
    credentials = service_account.Credentials.from_service_account_file(key_path)
    client = bigquery.Client(credentials=credentials, project=credentials.project_id)
    return client

In [4]:
def save_dataframe_to_bigquery(df, dataset_id, table_id, key_path):
    # BigQuery 클라이언트 객체 생성
    client = create_bigquery_client(key_path)

    # 테이블 레퍼런스 생성
    table_ref = client.dataset(dataset_id).table(table_id)

    # 데이터프레임을 BigQuery 테이블에 적재
    job_config = bigquery.LoadJobConfig()
    job_config.write_disposition = "WRITE_TRUNCATE"  # 기존 테이블 내용 삭제 후 삽입

    job = client.load_table_from_dataframe(df, table_ref, job_config=job_config)
    job.result()  # 작업 완료 대기

    print(f"Data inserted into table {table_id} successfully.")

In [5]:
def get_dataframe_from_bigquery(dataset_id, table_id, key_path):
    # BigQuery 클라이언트 생성
    client = create_bigquery_client(key_path)

    # 테이블 레퍼런스 생성
    table_ref = client.dataset(dataset_id).table(table_id)

    # 테이블 데이터를 DataFrame으로 변환
    df = client.list_rows(table_ref).to_dataframe()

    return df

In [6]:
# 오늘 날짜
today = datetime.today()#.strftime('%Y%m%d')

# 어제 날짜 계산
ytday = datetime.today() - timedelta(days=1)

# 만약 어제, 오늘이 토요일(5) 또는 일요일(6)이라면, 그 전주 금요일로 변경
if ytday.weekday() == 5:  # 토요일
    ytday -= timedelta(days=1)
elif ytday.weekday() == 6:  # 일요일
    ytday -= timedelta(days=2)
if today.weekday() == 5:  # 토요일
    today -= timedelta(days=1)
elif today.weekday() == 6:  # 일요일
    today -= timedelta(days=2)

# 'YYYYMMDD' 형식으로 변환
ytday = ytday.strftime('%Y%m%d')
today = today.strftime('%Y%m%d')

#### 법정동코드

In [7]:
dist_code = pd.read_csv('C:\py_src\midoproject\data/법정동코드 전체자료.csv',encoding='cp949')

In [8]:
dist_code_list = []
for i in range(dist_code['법정동명'].str.split().apply(len).max()):
    dist_code_temp = dist_code[dist_code['법정동명'].str.split().str[i].str[-1].isin(['읍','면','동','리','가','로']).dropna()]
    dist_code_list.append([dist_code_temp['법정동명'].str.split().str[i].dropna().unique()])

emd_code_list = list(chain(*chain(*dist_code_list)))

#### 종합쇼핑몰 납품상세내역

In [9]:
all_shop_df = get_dataframe_from_bigquery('g2b', 'shop_detail_df_all', bigquerykey_path)

In [10]:
all_shop_df_fin = all_shop_df[['납품요구접수일자', '수요기관명', '납품요구건명', '업체명', '단가', '단위', '수량', '금액', '수요기관코드', '수요기관구분', '수요기관지역명','납품요구지청명']]
all_shop_df_fin = all_shop_df_fin.sort_values(['납품요구접수일자'],ascending=False).reset_index(drop=True)

In [11]:
# 특수문자, 숫자, 영어 제거 함수 (연속된 공백을 단일 공백으로 변환 포함)
def clean_text(text):
    text = re.sub('[^ㄱ-ㅎㅏ-ㅣ가-힣\s]', ' ', text)
    text = re.sub('\s+', ' ', text).strip()  # 연속된 공백을 단일 공백으로 변환
    return text

In [12]:
# 사업명 전처리
all_shop_df_fin['납품요구건명_re'] = all_shop_df_fin['납품요구건명'].apply(clean_text)
# all_shop_df_fin['납품요구건명_re'] = all_shop_df_fin['납품요구건명_re'].str.replace('초등학교','').str.replace('중학교','').str.replace('고등학교','')
all_shop_df_fin['납품요구건명_re'] = all_shop_df_fin['납품요구건명_re'].str.strip()

all_shop_df_fin['납품요구건명_re'] = all_shop_df_fin['납품요구건명_re'].str.split().apply(lambda x: [kw for kw in x if len(kw) > 1]) ## 글자수 1 제외
all_shop_df_fin['납품요구건명_re'] = all_shop_df_fin['납품요구건명_re'].apply((' ').join)

In [13]:
# 수요기관 필터링
dist_nm = pd.Series(all_shop_df_fin[all_shop_df_fin['수요기관지역명'].str.split(' ').str[1].notnull()]['수요기관지역명'].str.split(' ').str[1].unique())
dist_nm = np.where(dist_nm.apply(len)<=2,dist_nm,dist_nm.str[:-1])

In [14]:
# 사업명 split 키워드 필터링
mapping_keywd_all = list(chain(*all_shop_df_fin['납품요구건명_re'].str.split(' '))) ## 전체 키워드
mapping_keywd_all_filter_cnt = pd.Series(Counter(mapping_keywd_all))

mapping_keywd3 = list(chain(*all_shop_df_fin['납품요구건명_re'].str.split(' ').str[:3])) ## split 3개 키워드
mapping_keywd3_filter_cnt = pd.Series(Counter(mapping_keywd3))

mapping_keywd2 = list(chain(*all_shop_df_fin['납품요구건명_re'].str.split(' ').str[:2])) ## split 2개 키워드
mapping_keywd2_filter_cnt = pd.Series(Counter(mapping_keywd2))

mapping_keywd1 = list(chain(*all_shop_df_fin['납품요구건명_re'].str.split(' ').str[:1])) ## split 1개 키워드
mapping_keywd1_filter_cnt = pd.Series(Counter(mapping_keywd1))

mapping_keywd_all_filter_cnt_nm = ('|').join(mapping_keywd_all_filter_cnt[mapping_keywd_all_filter_cnt <= mapping_keywd_all_filter_cnt.mean()].keys())
mapping_keywd3_filter_cnt_nm = ('|').join(mapping_keywd3_filter_cnt[mapping_keywd3_filter_cnt <= mapping_keywd3_filter_cnt.mean()].keys())
mapping_keywd2_filter_cnt_nm = ('|').join(mapping_keywd2_filter_cnt[mapping_keywd2_filter_cnt <= mapping_keywd2_filter_cnt.mean()].keys())
mapping_keywd1_filter_cnt_nm = ('|').join(mapping_keywd1_filter_cnt[mapping_keywd1_filter_cnt <= mapping_keywd1_filter_cnt.mean()].keys())

In [15]:
# 전체 키워드 데이터셋
mapping_keywd_df = pd.DataFrame(Counter(mapping_keywd_all).items(),columns=['키워드','빈도수'])
mapping_keywd_df = mapping_keywd_df[mapping_keywd_df['키워드'].apply(len)!=1].reset_index(drop=True)
mapping_keywd_df.sort_values('빈도수',ascending=False)

,키워드,빈도수
8,인조잔디,830
0,관급자재,467
2,구입,262
27,구매,140
7,운동장,100
...,...,...
593,해안경비단,1
594,해안,1
595,경비대,1
597,연경장,1


In [16]:
# 종합쇼핑몰 기준 중요키워드 추출
print('1등급 키워드 : ', list(mapping_keywd_df[mapping_keywd_df['빈도수'] >= 50].sort_values('빈도수',ascending=False)['키워드']))
print('2등급 키워드 : ', list(mapping_keywd_df[(mapping_keywd_df['빈도수'] < 50) & (mapping_keywd_df['빈도수'] >= 40)].sort_values('빈도수',ascending=False)['키워드']))
print('3등급 키워드 : ', list(mapping_keywd_df[(mapping_keywd_df['빈도수'] < 40) & (mapping_keywd_df['빈도수'] >= 30)].sort_values('빈도수',ascending=False)['키워드']))
print('4등급 키워드 : ', list(mapping_keywd_df[(mapping_keywd_df['빈도수'] < 30) & (mapping_keywd_df['빈도수'] >= 20)].sort_values('빈도수',ascending=False)['키워드']))
print('5등급 키워드 : ', list(mapping_keywd_df[(mapping_keywd_df['빈도수'] < 20) & (mapping_keywd_df['빈도수'] >= 10)].sort_values('빈도수',ascending=False)['키워드']))
print('기타 키워드 : ', list(mapping_keywd_df[(mapping_keywd_df['빈도수'] < 10)].sort_values('빈도수',ascending=False)['키워드']))

1등급 키워드 :  ['인조잔디', '관급자재', '구입', '구매', '운동장', '조성공사', '공사', '테니스장', '관급', '조성사업', '게이트볼장', '체육시설', '정비공사', '교체공사']
2등급 키워드 :  ['조성']
3등급 키워드 :  ['축구장', '설치', '풋살장', '개선공사']
4등급 키워드 :  ['파크골프장', '족구장', '설치공사', '교체', '토목', '조달구매']
5등급 키워드 :  ['보수공사', '환경개선', '환경개선공사', '정비사업', '개보수공사', '다목적구장', '건축', '시설개선사업', '개선사업', '시설개선', '시설개선공사', '잔디', '부대', '연병장', '보수', '조달', '개선', '다목적', '개소', '생활체육공원', '사업', '체육공원']
기타 키워드 :  ['관련', '물품', '단계', '교체사업', '생활체육시설', '신축공사', '정비', '조경', '어린이공원', '야구장', '개보수', '신축', '추가', '시설', '실내게이트볼장', '구장', '체육', '노후시설', '위한', '따른', '김영훈', '차분', '시행', '창녕국민체육센터', '요청', '트랙조성', '해운대수목원', '관급자', '치수과', '당진', '인조잔디운동장', '리모델링', '그라운드골프장', '구입의뢰', '본부', '공공하수처리시설', '증설공사', '배수판', '설치사업', '재정비사업', '배드민턴장', '일원', '여단', '대보수', '사단', '하부', '우수조달', '건립', '비가림시설', '학교운동장', '조경공사', '근린공원', '재정비공사', '관급자관급', '확장', '건축공사', '환경개선사업', '자재', '조달구입', '휀스', '환경친화적', '토목공사', '운동공간', '보조경기장', '트랙보수', '재조성', '옥외', '구매설치', '본원', '스포츠파크', '수지아르피아', '다목적체육관', '안빈', '인조잔디외', '공공체육시설', '운동

In [17]:
# 빈도수 키워드
freq_kwd = ('|').join(list(mapping_keywd_df[(mapping_keywd_df['빈도수'] < 10)]['키워드']))

#### 지자체 세부사업별 예산서

In [358]:
test = pd.read_csv('C:\py_src\midoproject\data/2024-07-10T00-34_export.csv')

In [359]:
# 빅쿼리에서 불러오기
budget_df_today = get_dataframe_from_bigquery('budget','budget_df_0' + today,bigquerykey_path)
budget_df_ytday = get_dataframe_from_bigquery('budget','budget_df_0' + ytday,bigquerykey_path)

In [360]:
budget_df_today = budget_df_today[budget_df_today['세부사업명'].isin(test['세부사업명'].unique())].reset_index(drop=True)

In [361]:
# 필요키워드1
need_kwd1 = '학교|초등학교|중학교|고등학교|대학교|다목적|운동장|종합운동장|공원|체육|체육공원|체육시설|체육센터|스포츠|스포츠센터|놀이시설|연병장|축구장|족구장|야구장|풋살장|배드민턴장|테니스장|게이트볼장|하키장'

# 필요키워드2
need_kwd2 = '개선|교체|보수|공사|구매|구입|설치|정비|조성'

# 필요키워드3
need_kwd3 = '인조잔디'

# 제외키워드
except_kwd = 'CCTV|cctv|LED|led|가로등|가속기|간접등|간판|개방|검진|경관|계단|골프장|관리|광고|교량|교실|교육|교차로|균열|그늘막|그림|급수관|급식|난방기|내진|냉방기|네트워크|노상|노점|농기계|단열|데이터|도로개설|도로방음벽|도로정비|도로확장|도서관|디자인|뚜껑|막구조물|맨홀|모노레일|모니터링|미끄럼|미세먼지|미술|방송|방수|방음|방음벽|배면도로|배수|버스|법률자문|벤치|벽화|보건|보도블럭|보상|보일러|보행자도로|복지|볼라드|분묘|불법|빗물받이|사용료|사워실|살포|생육환경|석면|선홈통|소프트웨어|수도|수로|수목표찰|수영장|스크린|스프링클러|승강기|시계탑|시스템|식수대|신호등|안심벨|안전|약수터|억제제|역량|연결도로|예방|예술|오염|옹벽|요양기관|운영|울타리|위생|위험수목|유아|육교|음성안내|음악|의자|인양기|인터넷|입학|자료|자전거|저장고|저장장치|전광판|전기|전신주|전통시장|정화|정화조|조도|조명|조형물|조화기|주차장|줄눈|진로진학|진입도로|창고|창출|처우|철거|취업|치안|카메라|카약|통학로|파고라|펜스|폐기물|폐열|포충기|풀장|풍차|하수|하수관로|하수도|하천|학습|행사|화장|화장실|활성화|횡단보도|휀스|흡연'

In [362]:
budget_df_today = budget_df_today[((budget_df_today['세부사업명'].str.contains(need_kwd1) & 
                                  budget_df_today['세부사업명'].str.contains(need_kwd2)) &
                                  ~budget_df_today['세부사업명'].str.contains(except_kwd)) |
                                  budget_df_today['세부사업명'].str.contains(need_kwd3)
                                  ].reset_index(drop=True)

In [23]:
# 종료 사업
budget_df_delete_temp1 = budget_df_ytday[budget_df_ytday['세부사업코드'].isin(set(budget_df_ytday['세부사업코드']) - set(budget_df_today['세부사업코드']))].reset_index(drop=True)
budget_df_delete_temp2 = budget_df_ytday[budget_df_ytday['세부사업명'].isin(set(budget_df_ytday['세부사업명']) - set(budget_df_today['세부사업명']))].reset_index(drop=True)
budget_df_delete = pd.concat([budget_df_delete_temp1,budget_df_delete_temp2],axis=0).drop_duplicates().reset_index(drop=True)
budget_df_delete

,회계연도,지역코드,지역명,자치단체코드,자치단체명,회계구분명,세부사업코드,세부사업명,집행일자,예산현액,지출액,편성액,분야명,부문명,행정자치단체코드
0,2024,1100000,서울,1100000,서울본청,일반회계,611000020133006A,한강공원 나들목 신설 및 개선,20240709,6339972620,565495620,2587720000,환경,환경보호일반,6110000
1,2024,1100000,서울,1100000,서울본청,도시개발특별회계,6110000202230317,서리풀공원 교량형 보행연결로 설치,20240709,1081690630,1067890350,0,환경,상하수도ㆍ수질,6110000
2,2024,1100000,서울,1100000,서울본청,일반회계,611000020223032C,한강공원 공연·휴게공간 조성,20240709,982827760,961000790,0,환경,환경보호일반,6110000
3,2024,1100000,서울,1100000,서울본청,일반회계,6110000202330158,한강공원 승강기 신설 및 개선,20240709,1090811440,695463680,400000000,환경,환경보호일반,6110000
4,2024,1100000,서울,1100000,서울본청,일반회계,6110000202330360,광나루한강공원 수영장 재조성,20240709,3170000000,603538380,3170000000,환경,환경보호일반,6110000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2518,2024,4900000,제주,4900000,제주본청,일반회계,6500000202430051,장기미집행 도시계획시설(평대공원) 조성사업,20240709,180000000,75600000,180000000,국토및지역개발,지역및도시,6500000
2519,2024,4900000,제주,4900000,제주본청,일반회계,6500000202430052,장기미집행 도시계획시설(두모공원) 조성사업,20240709,60000000,36101100,60000000,국토및지역개발,지역및도시,6500000
2520,2024,4900000,제주,4900000,제주본청,일반회계,6500000202430053,장기미집행 도시계획시설(고산공원) 조성사업,20240709,60000000,29500000,60000000,국토및지역개발,지역및도시,6500000
2521,2024,4900000,제주,4900000,제주본청,일반회계,6500000202430080,장기미집행 도시계획시설(새섬공원) 조성사업,20240709,200000000,182100670,200000000,국토및지역개발,지역및도시,6500000


In [24]:
# 새로 추가된 사업
budget_df_new_temp1 = budget_df_today[~budget_df_today['세부사업코드'].isin(budget_df_ytday['세부사업코드'])].reset_index(drop=True).reset_index(drop=True)
budget_df_new_temp2 = budget_df_today[~budget_df_today['세부사업명'].isin(budget_df_ytday['세부사업명'])].reset_index(drop=True).reset_index(drop=True)
budget_df_new = pd.concat([budget_df_new_temp1,budget_df_new_temp2],axis=0).drop_duplicates().reset_index(drop=True)
budget_df_new

,회계연도,지역코드,지역명,자치단체코드,자치단체명,회계구분명,세부사업코드,세부사업명,집행일자,예산현액,지출액,편성액,분야명,부문명,행정자치단체코드


In [363]:
# 사업명 전처리
budget_df_today['세부사업명_re'] = budget_df_today['세부사업명'].apply(clean_text)
# budget_df_today['세부사업명_re'] = budget_df_today['세부사업명_re'].str.replace('초등학교','').str.replace('중학교','').str.replace('고등학교','')
budget_df_today['세부사업명_re'] = budget_df_today['세부사업명_re'].str.strip()

budget_df_today['세부사업명_re'] = budget_df_today['세부사업명_re'].str.split().apply(lambda x: [kw for kw in x if len(kw) > 1]) ## 글자수 1 제외
budget_df_today['세부사업명_re'] = budget_df_today['세부사업명_re'].apply((' ').join)

In [364]:
# 해당 지역 추출
filtered_budget_df_temp1 = budget_df_today[budget_df_today['자치단체명'].str.contains(('|').join(dist_nm))].reset_index(drop=True)
filtered_budget_df_temp2 = budget_df_today[budget_df_today['자치단체명'].str.contains('본청')].reset_index(drop=True)

filtered_budget_df_today = pd.concat([filtered_budget_df_temp1,filtered_budget_df_temp2],axis=0).reset_index(drop=True)

#### 지자체 종합쇼핑몰 전처리

In [365]:
# Initialize the dictionary for split keywords
shop_split_kwd = {}
for i in range(all_shop_df_fin['납품요구건명_re'].str.split().apply(len).max()):
    shop_split_kwd['kwd_' + str(i)] = all_shop_df_fin['납품요구건명_re'].str.split().str[i]

In [366]:
# 매핑데이터셋
mapping_merge_df = []

In [367]:
# 종합쇼핑몰 납품요구건명 split 키워드 순서대로 맵핑
# Initialize the list to store the results
prep_budget_df_today = filtered_budget_df_today.copy()
prep_all_shop_df_fin = all_shop_df_fin.copy()

# Iterate over each keyword column
for i in range(len(shop_split_kwd)):
    keyword_column = 'kwd_' + str(i)
    
    # Get unique non-null keywords
    shop_split_kwd_temp = shop_split_kwd[keyword_column].dropna().unique()

    prep_budget_df_today[keyword_column] = prep_budget_df_today['세부사업명_re'].apply(lambda x: ' '.join(re.findall('|'.join(shop_split_kwd_temp), x)))
    prep_budget_df_today[keyword_column] = prep_budget_df_today[keyword_column].str.strip()
    prep_budget_df_today[keyword_column] = prep_budget_df_today[keyword_column].str.split().apply(lambda x : ' '.join(sorted(set(x)))).str.strip()

    prep_all_shop_df_fin[keyword_column] = prep_all_shop_df_fin['납품요구건명_re'].apply(lambda x: ' '.join(re.findall('|'.join(shop_split_kwd_temp), x)))
    prep_all_shop_df_fin[keyword_column] = prep_all_shop_df_fin[keyword_column].str.strip()
    prep_all_shop_df_fin[keyword_column] = prep_all_shop_df_fin[keyword_column].str.split().apply(lambda x : ' '.join(sorted(set(x)))).str.strip()

    mapping_merge_df_temp = pd.merge(
            prep_budget_df_today[(prep_budget_df_today[keyword_column] != '') & ~(prep_budget_df_today[keyword_column].isnull())][['지역명','자치단체명','세부사업코드','세부사업명', '세부사업명_re', keyword_column]].reset_index(drop=True),
            prep_all_shop_df_fin[(prep_all_shop_df_fin[keyword_column] != '') & ~(prep_all_shop_df_fin[keyword_column].isnull())][['수요기관명','수요기관지역명','납품요구건명', '납품요구건명_re', keyword_column]].reset_index(drop=True),
            how='left',
            on=keyword_column
            ).drop_duplicates().reset_index(drop=True)
    
    # Append the temporary DataFrame to the results list
    mapping_merge_df.append(mapping_merge_df_temp)

In [368]:
# 종합쇼핑몰 지역 키워드 맵핑 - 세부사업명 지역 및 해당지역 추출
# Initialize the columns for district keywords
prep_budget_df_today = filtered_budget_df_today.copy()
prep_all_shop_df_fin = all_shop_df_fin.copy()

sig_pat = '|'.join(dist_nm)

prep_budget_df_today['kwd_sig'] = prep_budget_df_today['세부사업명_re'].apply(lambda x: ' '.join(re.findall(sig_pat, x)))
prep_budget_df_today['kwd_sig'] = prep_budget_df_today['kwd_sig'].str.strip()
prep_budget_df_today['kwd_sig'] = prep_budget_df_today['kwd_sig'].str.split()
prep_budget_df_today = prep_budget_df_today.explode('kwd_sig')

prep_all_shop_df_fin['kwd_sig'] = prep_all_shop_df_fin['납품요구건명_re'].apply(lambda x: ' '.join(re.findall(sig_pat, x)))
prep_all_shop_df_fin['kwd_sig'] = prep_all_shop_df_fin['kwd_sig'].str.strip()
prep_all_shop_df_fin['kwd_sig'] = prep_all_shop_df_fin['kwd_sig'].str.split()
prep_all_shop_df_fin = prep_all_shop_df_fin.explode('kwd_sig')

mapping_merge_df_sig = pd.merge(
    prep_budget_df_today[(prep_budget_df_today['kwd_sig'] != '') & ~(prep_budget_df_today['kwd_sig'].isnull())][['지역명','자치단체명','세부사업코드','세부사업명', '세부사업명_re', 'kwd_sig']].reset_index(drop=True),
    prep_all_shop_df_fin[(prep_all_shop_df_fin['kwd_sig'] != '') & ~(prep_all_shop_df_fin['kwd_sig'].isnull())][['수요기관명','수요기관지역명','납품요구건명', '납품요구건명_re', 'kwd_sig']].reset_index(drop=True),
    how='left',
    on='kwd_sig'
    ).drop_duplicates().reset_index(drop=True)

# Append the district keywords DataFrame to the results list
mapping_merge_df.append(mapping_merge_df_sig)

In [369]:
# 종합쇼핑몰 지역 키워드 맵핑 - 세부사업명 지역 및 해당지역 읍면동 추출
# Initialize the columns for district keywords
prep_budget_df_today = filtered_budget_df_today.copy()
prep_all_shop_df_fin = all_shop_df_fin.copy()

emd_pat = '|'.join(set(emd_code_list))

prep_budget_df_today['kwd_emd'] = prep_budget_df_today['세부사업명_re'].apply(lambda x: ' '.join(re.findall(emd_pat, x)))
prep_budget_df_today['kwd_emd'] = prep_budget_df_today['kwd_emd'].str.strip()
prep_budget_df_today['kwd_emd'] = prep_budget_df_today['kwd_emd'].str.split()
prep_budget_df_today = prep_budget_df_today.explode('kwd_emd')

prep_all_shop_df_fin['kwd_emd'] = prep_all_shop_df_fin['납품요구건명_re'].apply(lambda x: ' '.join(re.findall(emd_pat, x)))
prep_all_shop_df_fin['kwd_emd'] = prep_all_shop_df_fin['kwd_emd'].str.strip()
prep_all_shop_df_fin['kwd_emd'] = prep_all_shop_df_fin['kwd_emd'].str.split()
prep_all_shop_df_fin = prep_all_shop_df_fin.explode('kwd_emd')

mapping_merge_df_emd = pd.merge(
    prep_budget_df_today[(prep_budget_df_today['kwd_emd'] != '') & ~(prep_budget_df_today['kwd_emd'].isnull())][['지역명','자치단체명','세부사업코드','세부사업명', '세부사업명_re', 'kwd_emd']].reset_index(drop=True),
    prep_all_shop_df_fin[(prep_all_shop_df_fin['kwd_emd'] != '') & ~(prep_all_shop_df_fin['kwd_emd'].isnull())][['수요기관명','수요기관지역명','납품요구건명', '납품요구건명_re', 'kwd_emd']].reset_index(drop=True),
    how='left',
    on='kwd_emd'
    ).drop_duplicates().reset_index(drop=True)

# Append the district keywords DataFrame to the results list
mapping_merge_df.append(mapping_merge_df_emd)

In [370]:
# 필요키워드 앞부분 추출
need_kwd_split = list(set(chain(*all_shop_df_fin['납품요구건명_re'].apply(lambda x: re.split(need_kwd1,x)).apply(''.join).str.split())))
need_kwd_split_list = list(pd.Series(need_kwd_split)[pd.Series(need_kwd_split).apply(len) >= 2])

In [371]:
# 종합쇼핑몰 지역 키워드 맵핑 - 필요키워드 앞부분 매핑
# Initialize the columns for district keywords
prep_budget_df_today = filtered_budget_df_today.copy()
prep_all_shop_df_fin = all_shop_df_fin.copy()

kwd_split_pat = '|'.join(set(need_kwd_split_list))

prep_budget_df_today['kwd_need_split'] = prep_budget_df_today['세부사업명_re'].apply(lambda x: ' '.join(re.findall(kwd_split_pat, x)))
prep_budget_df_today['kwd_need_split'] = prep_budget_df_today['kwd_need_split'].str.strip()
prep_budget_df_today['kwd_need_split'] = prep_budget_df_today['kwd_need_split'].str.split()
prep_budget_df_today = prep_budget_df_today.explode('kwd_need_split')

prep_all_shop_df_fin['kwd_need_split'] = prep_all_shop_df_fin['납품요구건명_re'].apply(lambda x: ' '.join(re.findall(kwd_split_pat, x)))
prep_all_shop_df_fin['kwd_need_split'] = prep_all_shop_df_fin['kwd_need_split'].str.strip()
prep_all_shop_df_fin['kwd_need_split'] = prep_all_shop_df_fin['kwd_need_split'].str.split()
prep_all_shop_df_fin = prep_all_shop_df_fin.explode('kwd_need_split')

mapping_merge_df_need_split = pd.merge(
    prep_budget_df_today[(prep_budget_df_today['kwd_need_split'] != '') & ~(prep_budget_df_today['kwd_need_split'].isnull())][['지역명','자치단체명','세부사업코드','세부사업명', '세부사업명_re', 'kwd_need_split']].reset_index(drop=True),
    prep_all_shop_df_fin[(prep_all_shop_df_fin['kwd_need_split'] != '') & ~(prep_all_shop_df_fin['kwd_need_split'].isnull())][['수요기관명','수요기관지역명','납품요구건명', '납품요구건명_re', 'kwd_need_split']].reset_index(drop=True),
    how='left',
    on='kwd_need_split'
    ).drop_duplicates().reset_index(drop=True)

# Append the district keywords DataFrame to the results list
mapping_merge_df.append(mapping_merge_df_need_split)

In [372]:
# 종합쇼핑몰 지역 키워드 맵핑 - 필요키워드 매핑
# Initialize the columns for district keywords
prep_budget_df_today = filtered_budget_df_today.copy()
prep_all_shop_df_fin = all_shop_df_fin.copy()

prep_budget_df_today['kwd_need'] = prep_budget_df_today['세부사업명_re'].apply(lambda x: ' '.join(re.findall(need_kwd1, x)))
prep_budget_df_today['kwd_need'] = prep_budget_df_today['kwd_need'].str.strip()
prep_budget_df_today['kwd_need'] = prep_budget_df_today['kwd_need'].str.split()
prep_budget_df_today = prep_budget_df_today.explode('kwd_need')

prep_all_shop_df_fin['kwd_need'] = prep_all_shop_df_fin['납품요구건명_re'].apply(lambda x: ' '.join(re.findall(need_kwd1, x)))
prep_all_shop_df_fin['kwd_need'] = prep_all_shop_df_fin['kwd_need'].str.strip()
prep_all_shop_df_fin['kwd_need'] = prep_all_shop_df_fin['kwd_need'].str.split()
prep_all_shop_df_fin = prep_all_shop_df_fin.explode('kwd_need')

mapping_merge_df_need = pd.merge(
    prep_budget_df_today[(prep_budget_df_today['kwd_need'] != '') & ~(prep_budget_df_today['kwd_need'].isnull())][['지역명','자치단체명','세부사업코드','세부사업명', '세부사업명_re', 'kwd_need']].reset_index(drop=True),
    prep_all_shop_df_fin[(prep_all_shop_df_fin['kwd_need'] != '') & ~(prep_all_shop_df_fin['kwd_need'].isnull())][['수요기관명','수요기관지역명','납품요구건명', '납품요구건명_re', 'kwd_need']].reset_index(drop=True),
    how='left',
    on='kwd_need'
    ).drop_duplicates().reset_index(drop=True)

# Append the district keywords DataFrame to the results list
mapping_merge_df.append(mapping_merge_df_need)

In [373]:
# 키워드별 데이터셋 병합
mapping_merge_df_fin = pd.concat(mapping_merge_df, axis=0).drop_duplicates()
mapping_merge_df_fin = mapping_merge_df_fin[~mapping_merge_df_fin['납품요구건명'].isnull()].reset_index(drop=True)

In [374]:
# 모든 키워드 합
kwd_columns = ['kwd_0','kwd_1','kwd_2','kwd_3','kwd_4','kwd_5','kwd_6','kwd_7','kwd_8','kwd_sig','kwd_emd','kwd_need_split','kwd_need']
mapping_merge_df_fin['all_kwd'] = mapping_merge_df_fin[kwd_columns].apply(lambda x : (' ').join(x.dropna()),axis=1)
mapping_merge_df_fin['all_kwd'] = mapping_merge_df_fin['all_kwd'].str.strip()
mapping_merge_df_fin['all_kwd'] = mapping_merge_df_fin['all_kwd'].str.split().apply(lambda x : ' '.join(sorted(set(x)))).str.strip()
mapping_merge_df_fin.drop(columns=kwd_columns, inplace=True)

In [375]:
# 지역명 추출1
mapping_merge_df_fin['수요기관지역명_split1'] = mapping_merge_df_fin['수요기관지역명'].str.split().str[0]

# Define the replacements
replacements = {
    '전라북도': '전북',
    '전라남도': '전남',
    '경상북도': '경북',
    '경상남도': '경남',
    '충청북도': '충북',
    '충청남도': '충남'
}

# Apply the replacements
mapping_merge_df_fin['수요기관지역명_split1'] = mapping_merge_df_fin['수요기관지역명_split1'].replace(replacements)
mapping_merge_df_fin['수요기관지역명_split1'] = mapping_merge_df_fin['수요기관지역명_split1'].str[:2]

In [376]:
# 지역명 추출2
mapping_merge_df_fin['수요기관지역명_split2'] = mapping_merge_df_fin['수요기관지역명'].str.split().str[1]
dist_sig_pat = '|'.join(mapping_merge_df_fin['수요기관지역명_split2'].dropna().unique())

mapping_merge_df_fin['자치단체명_re'] = mapping_merge_df_fin['자치단체명'].apply(lambda x: ' '.join(re.findall(dist_sig_pat, x)))
mapping_merge_df_fin['자치단체명_re'] = mapping_merge_df_fin['자치단체명_re'].str.strip()
mapping_merge_df_fin['자치단체명_re'] = mapping_merge_df_fin['자치단체명_re'].str.split()
mapping_merge_df_fin = mapping_merge_df_fin.explode('자치단체명_re')

In [377]:
# 제외 지역 제거
filtered_mapping_merge_df_fin = mapping_merge_df_fin[mapping_merge_df_fin['자치단체명_re']!=''].drop_duplicates().reset_index(drop=True)

In [378]:
# 지역 일치 확인1
mapping_merge_df_fin_temp1 = filtered_mapping_merge_df_fin[filtered_mapping_merge_df_fin['지역명']==filtered_mapping_merge_df_fin['수요기관지역명_split1']].drop_duplicates().reset_index(drop=True)
mapping_merge_df_fin_temp1 = mapping_merge_df_fin_temp1[mapping_merge_df_fin_temp1['자치단체명'].str.contains('|'.join(mapping_merge_df_fin_temp1['수요기관지역명_split2'].dropna().unique()))].drop_duplicates().reset_index(drop=True)
mapping_merge_df_fin_temp1 = mapping_merge_df_fin_temp1[mapping_merge_df_fin_temp1['자치단체명_re']==mapping_merge_df_fin_temp1['수요기관지역명_split2']].drop_duplicates().reset_index(drop=True)

# 본청 포함 확인
mapping_merge_df_fin_temp2 = filtered_mapping_merge_df_fin[filtered_mapping_merge_df_fin['자치단체명'].str.contains('본청')].drop_duplicates().reset_index(drop=True)

# 데이터 병합
mapping_merge_df_final = pd.concat([mapping_merge_df_fin_temp1,mapping_merge_df_fin_temp2], axis=0).drop_duplicates().reset_index(drop=True)

In [646]:
# 그룹화할 컬럼 리스트
group_cols = ['지역명', '자치단체명', '세부사업코드', '세부사업명', '세부사업명_re', '수요기관명', '수요기관지역명', '납품요구건명', '납품요구건명_re','수요기관지역명_split1', '자치단체명_re', '수요기관지역명_split2']

# 그룹화 및 all_kwd 컬럼 조인
mapping_merge_df_final_grp = mapping_merge_df_final.groupby(group_cols).agg({'all_kwd': ' '.join}).reset_index()
mapping_merge_df_final_grp['all_kwd'] = mapping_merge_df_final_grp['all_kwd'].str.strip()
mapping_merge_df_final_grp['all_kwd'] = mapping_merge_df_final_grp['all_kwd'].str.split().apply(lambda x : ' '.join(sorted(set(x)))).str.strip()

In [647]:
# 필요키워드 기준
mapping_merge_df_final_grp['세부사업명_키워드1'] = mapping_merge_df_final_grp['세부사업명'].apply(lambda x: ' '.join(re.findall(need_kwd1, x)))
mapping_merge_df_final_grp['납품요구건명_키워드1'] = mapping_merge_df_final_grp['납품요구건명'].apply(lambda x: ' '.join(re.findall(need_kwd1, x)))

# 사업명 지역 추출
mapping_merge_df_final_grp['세부사업명_지역'] = mapping_merge_df_final_grp['세부사업명'].apply(lambda x: ' '.join(re.findall(emd_pat, x)))
mapping_merge_df_final_grp['납품요구건명_지역'] = mapping_merge_df_final_grp['납품요구건명'].apply(lambda x: ' '.join(re.findall(emd_pat, x)))

# 필요키워드 패턴 기준
need_kwd_pat = '\w+' + ('|\w+').join(need_kwd1.split('|'))

mapping_merge_df_final_grp['세부사업명_키워드2'] = mapping_merge_df_final_grp['세부사업명'].apply(lambda x: ' '.join(re.findall(need_kwd_pat, x)))
mapping_merge_df_final_grp['납품요구건명_키워드2'] = mapping_merge_df_final_grp['납품요구건명'].apply(lambda x: ' '.join(re.findall(need_kwd_pat, x)))

In [648]:
def check_inclusion1(row):
    set1 = set(row['세부사업명_키워드1'])
    set2 = set(row['납품요구건명_키워드1'])
    return bool(set1 & set2)

In [649]:
# 필요키워드 모두 없는것 제외
mapping_merge_df_final_grp_temp1 = mapping_merge_df_final_grp[(mapping_merge_df_final_grp['세부사업명_키워드1']!='')&(mapping_merge_df_final_grp['납품요구건명_키워드1']!='')].reset_index(drop=True)

# 필요키워드 포함여부 확인
mapping_merge_df_final_grp_temp1['세부사업명_키워드1'] = mapping_merge_df_final_grp_temp1['세부사업명_키워드1'].str.split() ## 키워드별 정리
mapping_merge_df_final_grp_temp1['납품요구건명_키워드1'] = mapping_merge_df_final_grp_temp1['납품요구건명_키워드1'].str.split() ## 키워드별 정리

mapping_merge_df_final_grp_temp1 = mapping_merge_df_final_grp_temp1[mapping_merge_df_final_grp_temp1.apply(check_inclusion1,axis=1)].reset_index(drop=True)

mapping_merge_df_final_grp_temp1['세부사업명_키워드1'] = mapping_merge_df_final_grp_temp1['세부사업명_키워드1'].apply(lambda x : ' '.join(sorted(set(x)))).str.strip()
mapping_merge_df_final_grp_temp1['납품요구건명_키워드1'] = mapping_merge_df_final_grp_temp1['납품요구건명_키워드1'].apply(lambda x : ' '.join(sorted(set(x)))).str.strip()

In [650]:
def check_inclusion_dist(row):
    set1 = set(row['세부사업명_지역'])
    set2 = set(row['납품요구건명_지역'])
    return bool(set1 & set2)

In [651]:
# 지역 없는것 제외
mapping_merge_df_final_grp_temp2 = mapping_merge_df_final_grp[(mapping_merge_df_final_grp['세부사업명_지역']!='')&(mapping_merge_df_final_grp['납품요구건명_지역']!='')].reset_index(drop=True)

# 사업명 지역 포함여부 확인
mapping_merge_df_final_grp_temp2['세부사업명_지역'] = mapping_merge_df_final_grp_temp2['세부사업명_지역'].str.split()
mapping_merge_df_final_grp_temp2['납품요구건명_지역'] = mapping_merge_df_final_grp_temp2['납품요구건명_지역'].str.split()

mapping_merge_df_final_grp_temp2 = mapping_merge_df_final_grp_temp2[mapping_merge_df_final_grp_temp2.apply(check_inclusion_dist,axis=1)].reset_index(drop=True)

mapping_merge_df_final_grp_temp2['세부사업명_지역'] = mapping_merge_df_final_grp_temp2['세부사업명_지역'].apply(lambda x : ' '.join(sorted(set(x)))).str.strip() ## 키워드별 정리
mapping_merge_df_final_grp_temp2['납품요구건명_지역'] = mapping_merge_df_final_grp_temp2['납품요구건명_지역'].apply(lambda x : ' '.join(sorted(set(x)))).str.strip() ## 키워드별 정리

In [652]:
def check_inclusion2(row):
    set1 = set(row['세부사업명_키워드2'])
    set2 = set(row['납품요구건명_키워드2'])
    return bool(set1 & set2)

In [653]:
# 필요키워드패턴 모두 없는것 제외
mapping_merge_df_final_grp_temp3 = mapping_merge_df_final_grp[(mapping_merge_df_final_grp['세부사업명_키워드2']!='')&(mapping_merge_df_final_grp['납품요구건명_키워드2']!='')].reset_index(drop=True)

# 필요키워드 패턴 포함여부 확인
mapping_merge_df_final_grp_temp3['세부사업명_키워드2'] = mapping_merge_df_final_grp_temp3['세부사업명_키워드2'].str.split() ## 키워드별 정리
mapping_merge_df_final_grp_temp3['납품요구건명_키워드2'] = mapping_merge_df_final_grp_temp3['납품요구건명_키워드2'].str.split() ## 키워드별 정리

mapping_merge_df_final_grp_temp3 = mapping_merge_df_final_grp_temp3[mapping_merge_df_final_grp_temp3.apply(check_inclusion2,axis=1)].reset_index(drop=True)

mapping_merge_df_final_grp_temp3['세부사업명_키워드2'] = mapping_merge_df_final_grp_temp3['세부사업명_키워드2'].apply(lambda x : ' '.join(sorted(set(x)))).str.strip()
mapping_merge_df_final_grp_temp3['납품요구건명_키워드2'] = mapping_merge_df_final_grp_temp3['납품요구건명_키워드2'].apply(lambda x : ' '.join(sorted(set(x)))).str.strip()

In [654]:
# 필터링 데이터셋 병합
mapping_merge_df_final_grp_fin = pd.concat([mapping_merge_df_final_grp_temp1,mapping_merge_df_final_grp_temp2,mapping_merge_df_final_grp_temp3]).drop_duplicates().reset_index(drop=True)
mapping_merge_df_final_grp_fin

,지역명,자치단체명,세부사업코드,세부사업명,세부사업명_re,수요기관명,수요기관지역명,납품요구건명,납품요구건명_re,수요기관지역명_split1,자치단체명_re,수요기관지역명_split2,all_kwd,세부사업명_키워드1,납품요구건명_키워드1,세부사업명_지역,납품요구건명_지역,세부사업명_키워드2,납품요구건명_키워드2
0,강원,강원동해시,4211000202430032,발한게이트볼장 조성사업(전환사업),발한게이트볼장 조성사업 전환사업,강원특별자치도 동해시,강원도 동해시,샘실8단지게이트볼장 인조잔디 구입 및 설치,샘실 단지게이트볼장 인조잔디 구입 설치,강원,동해시,동해시,게이트볼장,게이트볼장,게이트볼장,,,발한게이트볼장,샘실8단지게이트볼장
1,강원,강원삼척시,42400002023302CB,도계 전천후 테니스장 조성사업(도)(전환사업),도계 전천후 테니스장 조성사업 전환사업,강원특별자치도 삼척시,강원도 삼척시,도계늑구테니스장정비공사관급자재[인조잔디],도계늑구테니스장정비공사관급자재 인조잔디,강원,삼척시,삼척시,테니스 테니스장,테니스장,관급 테니스장,,,,도계늑구테니스장 정비공사관급
2,강원,강원원주시,41900002022301B9,금대2리 게이트볼장 조성사업,금대 게이트볼장 조성사업,강원특별자치도 원주시,강원특별자치도 원주시,관급자재 인조잔디 구매-신림게이트볼장 인조잔디 교체공사,관급자재 인조잔디 구매 신림게이트볼장 인조잔디 교체공사,강원,원주시,원주시,게이트볼장,게이트볼장,게이트볼장 관급,,,,신림게이트볼장
3,강원,강원원주시,41900002022301D4,태장체육단지 풋살장 막구조 설치,태장체육단지 풋살장 막구조 설치,강원특별자치도교육청 강원특별자치도원주교육지원청 원주삼육중학교,강원도 원주시,원주삼육중학교 풋살장 개보수 인조잔디 구입,원주삼육중학교 풋살장 개보수 인조잔디 구입,강원,원주시,원주시,풋살 풋살장,풋살장,중학교 풋살장,,,,원주삼육중학교
4,강원,강원원주시,4190000202230218,혁신도시 봉두공원 생활체육시설 조성,혁신도시 봉두공원 생활체육시설 조성,국방부 육군 제3139부대,강원도 원주시,00부대 체육시설 개선(관급자재),부대 체육시설 개선 관급자재,강원,원주시,원주시,시설 체육 체육시설,공원 체육시설,관급 체육시설,,,봉두공원 생활체육시설,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
317,경남,경남남해군,5430000202230020,남해 생활체육공원 조성사업,남해 생활체육공원 조성사업,경상남도 남해군,경상남도 남해군,창선 생활체육공원 운동장 육상트랙 교체공사 관급자재-인조잔디,창선 생활체육공원 운동장 육상트랙 교체공사 관급자재 인조잔디,경남,남해군,남해군,생활 체육 체육공원,체육공원,체육공원 운동장 관급,,,생활체육공원,생활체육공원
318,광주,광주광산구,36300002024300BF,첨단종합운동장 축구장 시설개선사업,첨단종합운동장 축구장 시설개선사업,광주광역시 광산구,광주광역시 광산구,용동근린공원 축구장 시설개선공사 관급자재 구입,용동근린공원 축구장 시설개선공사 관급자재 구입,광주,광산구,광산구,축구 축구장,종합운동장 축구장,공원 축구장 관급,,용동,첨단종합운동장 축구장,용동근린공원 축구장
319,전남,전남영광군,497000020243009D,대마산업단지 야구장 정비사업(2단계 도 전환사업)(전환사업),대마산업단지 야구장 정비사업 단계 전환사업 전환사업,전라남도 영광군,전라남도 영광군,대마산단 야구장 인조잔디 설치 및 부대시설 정비공사 관급자재 인조잔디 구,대마산단 야구장 인조잔디 설치 부대시설 정비공사 관급자재 인조잔디,전남,영광군,영광군,야구장 정비,야구장,야구장 관급,,,야구장,야구장
320,전북,전북전주시,4640000202330718,공공체육시설 축구장 인조잔디 교체,공공체육시설 축구장 인조잔디 교체,전북특별자치도 전주시,전북특별자치도 전주시 완산구,완산생활체육공원 축구장 인조잔디 교체공사 관급자재(인조잔디) 구입,완산생활체육공원 축구장 인조잔디 교체공사 관급자재 인조잔디 구입,전북,전주시,전주시,인조 잔디 체육 축구 축구장,체육시설 축구장,체육공원 축구장 관급,,,공공체육시설 축구장,완산생활체육공원 축구장


In [655]:
# 모든 키워드 합 중복 제거
kwd_columns = ['all_kwd','세부사업명_키워드1', '납품요구건명_키워드1', '세부사업명_지역', '납품요구건명_지역', '세부사업명_키워드2', '납품요구건명_키워드2']
mapping_merge_df_final_grp_fin['all_kwd_fin'] = mapping_merge_df_final_grp_fin[kwd_columns].apply(lambda x : (' ').join(x.dropna()),axis=1)
mapping_merge_df_final_grp_fin['all_kwd_fin'] = mapping_merge_df_final_grp_fin['all_kwd_fin'].str.strip()
mapping_merge_df_final_grp_fin['all_kwd_fin'] = mapping_merge_df_final_grp_fin['all_kwd_fin'].str.split().apply(lambda x : ' '.join(sorted(set(x)))).str.strip()
# mapping_merge_df_final_grp_fin.drop(columns=kwd_columns, inplace
# =True)

mapping_merge_df_final_grp_fin = mapping_merge_df_final_grp_fin.drop_duplicates(['지역명', '자치단체명', '세부사업코드', '세부사업명', '세부사업명_re', '수요기관명', '수요기관지역명',
                                                                                 '납품요구건명', '납품요구건명_re', '수요기관지역명_split1', '자치단체명_re', '수요기관지역명_split2', 'all_kwd_fin']).reset_index(drop=True)

In [656]:
# 최종필터링1 - 지역 일치
mapping_merge_df_final_grp_fin_temp1 = mapping_merge_df_final_grp_fin[(mapping_merge_df_final_grp_fin['세부사업명_지역']!='') & (mapping_merge_df_final_grp_fin['납품요구건명_지역']!='')]
mapping_merge_df_final_grp_fin_temp1 = mapping_merge_df_final_grp_fin_temp1[mapping_merge_df_final_grp_fin_temp1['세부사업명_지역']==mapping_merge_df_final_grp_fin_temp1['납품요구건명_지역']]

In [657]:
# 최종필터링2 - 필수키워드 패턴 포함 & 지역 불일치 제거
mapping_merge_df_final_grp_fin_temp2 = mapping_merge_df_final_grp_fin[(mapping_merge_df_final_grp_fin['세부사업명_키워드2']!='')&(mapping_merge_df_final_grp_fin['납품요구건명_키워드2']!='')].reset_index(drop=True)

# 필요키워드 패턴 포함여부 확인
mapping_merge_df_final_grp_fin_temp2['세부사업명_키워드2'] = mapping_merge_df_final_grp_fin_temp2['세부사업명_키워드2'].str.split() ## 키워드별 정리
mapping_merge_df_final_grp_fin_temp2['납품요구건명_키워드2'] = mapping_merge_df_final_grp_fin_temp2['납품요구건명_키워드2'].str.split() ## 키워드별 정리

mapping_merge_df_final_grp_fin_temp2 = mapping_merge_df_final_grp_fin_temp2[mapping_merge_df_final_grp_fin_temp2.apply(check_inclusion2,axis=1)].reset_index(drop=True)

mapping_merge_df_final_grp_fin_temp2['세부사업명_키워드2'] = mapping_merge_df_final_grp_fin_temp2['세부사업명_키워드2'].apply(lambda x : ' '.join(sorted(set(x)))).str.strip()
mapping_merge_df_final_grp_fin_temp2['납품요구건명_키워드2'] = mapping_merge_df_final_grp_fin_temp2['납품요구건명_키워드2'].apply(lambda x : ' '.join(sorted(set(x)))).str.strip()

# 지역 불일치 제거
mapping_merge_df_final_grp_fin_temp2 = mapping_merge_df_final_grp_fin_temp2[mapping_merge_df_final_grp_fin_temp2['세부사업명_지역']==mapping_merge_df_final_grp_fin_temp2['납품요구건명_지역']]

In [661]:
# 최종 필터링 데이터셋
mapping_merge_df_final_grp_final = pd.concat([mapping_merge_df_final_grp_fin_temp1,mapping_merge_df_final_grp_fin_temp2]).drop_duplicates().reset_index(drop=True)
mapping_merge_df_final_grp_final.drop(columns = 'all_kwd', inplace=True)

In [663]:
mapping_merge_df_final_grp_final.to_csv('C:\py_src\midoproject\data/filtered_mapping_df.csv',encoding='utf-8-sig',index=False)

In [687]:
# 종합쇼핑몰 기준 필터링
bid_cd1 = list(budget_df_today[budget_df_today['세부사업코드'].isin(mapping_merge_df_final_grp_final['세부사업코드'])]['세부사업코드'])
bid_cd2 = list(budget_df_today[budget_df_today['세부사업명'].isin(mapping_merge_df_final_grp_final['세부사업명'])]['세부사업코드'])

In [688]:
budget_df_today_fin = budget_df_today[~budget_df_today['세부사업코드'].isin(set(bid_cd1 + bid_cd2))].drop_duplicates().reset_index(drop=True)

In [689]:
# 중요 사업 체크
budget_df_today_final = pd.concat([budget_df_today_fin[budget_df_today_fin['세부사업명'].str.contains('인조잔디')],
                                   budget_df_today_fin[~budget_df_today_fin['세부사업명'].str.contains('인조잔디')]],axis=0).reset_index(drop=True)

# budget_df_ytday_final = pd.concat([budget_df_ytday_fin[budget_df_ytday_fin['세부사업명'].str.contains('인조잔디')],
#                                    budget_df_ytday_fin[~budget_df_ytday_fin['세부사업명'].str.contains('인조잔디')]],axis=0).reset_index(drop=True)

In [27]:
# 최종데이터셋
budget_df_today_final = budget_df_today_final[['회계연도', '집행일자', '지역명', '자치단체명', '회계구분명', '세부사업명','예산현액', '지출액', '편성액', '분야명', '부문명']].drop_duplicates().reset_index(drop=True)
budget_df_delete = budget_df_delete[['회계연도', '집행일자', '지역명', '자치단체명', '회계구분명', '세부사업명','예산현액', '지출액', '편성액', '분야명', '부문명']].drop_duplicates().reset_index(drop=True)
budget_df_new = budget_df_new[['회계연도', '집행일자', '지역명', '자치단체명', '회계구분명', '세부사업명','예산현액', '지출액', '편성액', '분야명', '부문명']].drop_duplicates().reset_index(drop=True)

In [29]:
# 빅쿼리에 적재
save_dataframe_to_bigquery(budget_df_today_final,'budget','budget_df_listup',bigquerykey_path)
save_dataframe_to_bigquery(budget_df_delete,'budget','budget_df_delete',bigquerykey_path)
save_dataframe_to_bigquery(budget_df_new,'budget','budget_df_new',bigquerykey_path)

Data inserted into table budget_df_listup successfully.
Data inserted into table budget_df_delete successfully.
Data inserted into table budget_df_new successfully.


#### 교육청 예산서

In [691]:
# 종합쇼핑몰 학교 관련 사업
shop_edu_mapping_temp1 = all_shop_df_fin[all_shop_df_fin['납품요구건명'].str.contains('학교|교육|교육청|교육지원청|교육부|공립|사립|국립')]
shop_edu_mapping_temp2 = all_shop_df_fin[all_shop_df_fin['수요기관명'].str.contains('학교|교육|교육청|교육지원청|교육부|공립|사립|국립')]
shop_edu_mapping_df = pd.concat([shop_edu_mapping_temp1,shop_edu_mapping_temp2],axis=0).drop_duplicates().reset_index(drop=True)
shop_edu_mapping_df

,납품요구접수일자,수요기관명,납품요구건명,업체명,단가,단위,수량,금액,수요기관코드,수요기관구분,수요기관지역명,납품요구지청명,납품요구건명_re
0,2024-07-09,대구광역시남부교육청 성지중학교,성지중학교 운동장 인조잔디( t55mm) 구입,주식회사 플랜에이,63200,㎡,2727.0,172346400,7270193,교육기관,대구광역시 달서구,대구지방조달청,성지중학교 운동장 인조잔디 구입
1,2024-07-09,보건복지부 국립소록도병원,국립소록도병원 테니스장 보수공사 관급자재(인조잔디),주식회사 플랜에이,38000,㎡,663.0,25194000,1352331,국가기관,전라남도 고흥군,광주지방조달청,국립소록도병원 테니스장 보수공사 관급자재 인조잔디
2,2024-07-05,경기도교육청 운산고등학교,운산고등학교 인조잔디 구매,주식회사 이원,63800,㎡,2985.0,190443000,7530982,교육기관,경기도 광명시,인천지방조달청,운산고등학교 인조잔디 구매
3,2024-07-05,부산광역시남부교육청 문현초등학교,문현초등학교 운동장 인조잔디,주식회사 유니스포텍,62900,㎡,2338.0,147060200,7180080,교육기관,부산광역시 남구,부산지방조달청,문현초등학교 운동장 인조잔디
4,2024-07-05,부산광역시해운대교육청 재송중학교,재송중학교 인조잔디 및 배수판 구입,주식회사 이원,63800,㎡,2161.0,137871800,7210042,교육기관,부산광역시 해운대구,부산지방조달청,재송중학교 인조잔디 배수판 구입
...,...,...,...,...,...,...,...,...,...,...,...,...,...
244,2024-01-09,서울특별시강동교육청 서울가동초등학교,실내운동장 인조잔디 구입,주식회사 필드글로벌,63000,㎡,495.0,31185000,7070052,교육기관,서울특별시 송파구,서울지방조달청,실내운동장 인조잔디 구입
245,2024-01-08,제주특별자치도교육청 제주시교육지원청 동광초등학교,동광초 인조잔디 및 울타리 조성공사 관급자재 구입(인조잔디 외 1종),주식회사 필드글로벌,63000,㎡,2680.0,168840000,9296060,교육기관,제주특별자치도 제주시,제주지방조달청,동광초 인조잔디 울타리 조성공사 관급자재 구입 인조잔디
246,2024-01-04,한국과학기술원부설한국과학영재학교,풋살장 조성 및 기타공사 관급자재 구매,주식회사 이원,38000,㎡,747.0,28386000,B552516,기타공공기관,부산광역시 부산진구,부산지방조달청,풋살장 조성 기타공사 관급자재 구매
247,2024-01-03,경기도파주교육청 파주중학교,파주중 인조잔디 운동장 조성사업 관급자재 구입,주식회사 지에스케이,61500,㎡,2540.0,156210000,7680017,교육기관,경기도 파주시,서울지방조달청,파주중 인조잔디 운동장 조성사업 관급자재 구입


In [692]:
# 교육청, 교육지원청, 학교명 필터링
shop_edu_mapping_df['수요기관명_re'] = shop_edu_mapping_df['수요기관명'].str.replace('교육청','').str.replace('교육지원청','')
shop_edu_mapping_df

,납품요구접수일자,수요기관명,납품요구건명,업체명,단가,단위,수량,금액,수요기관코드,수요기관구분,수요기관지역명,납품요구지청명,납품요구건명_re,수요기관명_re
0,2024-07-09,대구광역시남부교육청 성지중학교,성지중학교 운동장 인조잔디( t55mm) 구입,주식회사 플랜에이,63200,㎡,2727.0,172346400,7270193,교육기관,대구광역시 달서구,대구지방조달청,성지중학교 운동장 인조잔디 구입,대구광역시남부 성지중학교
1,2024-07-09,보건복지부 국립소록도병원,국립소록도병원 테니스장 보수공사 관급자재(인조잔디),주식회사 플랜에이,38000,㎡,663.0,25194000,1352331,국가기관,전라남도 고흥군,광주지방조달청,국립소록도병원 테니스장 보수공사 관급자재 인조잔디,보건복지부 국립소록도병원
2,2024-07-05,경기도교육청 운산고등학교,운산고등학교 인조잔디 구매,주식회사 이원,63800,㎡,2985.0,190443000,7530982,교육기관,경기도 광명시,인천지방조달청,운산고등학교 인조잔디 구매,경기도 운산고등학교
3,2024-07-05,부산광역시남부교육청 문현초등학교,문현초등학교 운동장 인조잔디,주식회사 유니스포텍,62900,㎡,2338.0,147060200,7180080,교육기관,부산광역시 남구,부산지방조달청,문현초등학교 운동장 인조잔디,부산광역시남부 문현초등학교
4,2024-07-05,부산광역시해운대교육청 재송중학교,재송중학교 인조잔디 및 배수판 구입,주식회사 이원,63800,㎡,2161.0,137871800,7210042,교육기관,부산광역시 해운대구,부산지방조달청,재송중학교 인조잔디 배수판 구입,부산광역시해운대 재송중학교
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
244,2024-01-09,서울특별시강동교육청 서울가동초등학교,실내운동장 인조잔디 구입,주식회사 필드글로벌,63000,㎡,495.0,31185000,7070052,교육기관,서울특별시 송파구,서울지방조달청,실내운동장 인조잔디 구입,서울특별시강동 서울가동초등학교
245,2024-01-08,제주특별자치도교육청 제주시교육지원청 동광초등학교,동광초 인조잔디 및 울타리 조성공사 관급자재 구입(인조잔디 외 1종),주식회사 필드글로벌,63000,㎡,2680.0,168840000,9296060,교육기관,제주특별자치도 제주시,제주지방조달청,동광초 인조잔디 울타리 조성공사 관급자재 구입 인조잔디,제주특별자치도 제주시 동광초등학교
246,2024-01-04,한국과학기술원부설한국과학영재학교,풋살장 조성 및 기타공사 관급자재 구매,주식회사 이원,38000,㎡,747.0,28386000,B552516,기타공공기관,부산광역시 부산진구,부산지방조달청,풋살장 조성 기타공사 관급자재 구매,한국과학기술원부설한국과학영재학교
247,2024-01-03,경기도파주교육청 파주중학교,파주중 인조잔디 운동장 조성사업 관급자재 구입,주식회사 지에스케이,61500,㎡,2540.0,156210000,7680017,교육기관,경기도 파주시,서울지방조달청,파주중 인조잔디 운동장 조성사업 관급자재 구입,경기도파주 파주중학교


In [693]:
shop_edu_mapping_df[shop_edu_mapping_df['수요기관명_re'].str.split().str[0].notnull()]['수요기관명_re'].str.split().str[0].unique()

array(['대구광역시남부', '보건복지부', '경기도', '부산광역시남부', '부산광역시해운대', '서울특별시동작',
       '전북특별자치도', '서울특별시남부', '전라남도', '선린대학', '서울특별시동부', '인천광역시남부',
       '경기도용인', '충청남도', '대구광역시', '서울특별시성동', '서울특별시', '강원특별자치도', '대전광역시',
       '경상북도', '서울특별시북부', '서울특별시서부', '경상남도거제', '경상남도진주', '경상남도',
       '서울특별시강동', '서울특별시중부', '전북대학교', '산림청', '동명대학교', '충청남도보령', '해양수산부',
       '인천광역시', '경기도김포', '공주대학교', '전라남도해남', '울산광역시', '경기도평택', '부산광역시',
       '경기도수원', '인천광역시강화', '서울특별시강서', '연암공과대학교산학협력단', '부산광역시북부',
       '강릉원주대학교', '부경대학교', '제주특별자치도', '목포해양대학교', '춘천교육대학교', '서울특별시강남',
       '충청북도', '육군학생군사학교', '통일부', '광주광역시', '서울특별시성북', '육군부사관학교', '목포대학교',
       '부산광역시동래', '교육부', '강원대학교', '경찰청', '청주교육대학교', '전남대학교', '광주교육대학교',
       '한국해양대학교', '경기도연천', '경기도포천', '경상남도밀양', '한국과학기술원부설한국과학영재학교',
       '경기도파주'], dtype=object)

In [389]:
# 교육청, 교육지원청, 학교명 필터링
shop_edu_mapping_df['수요기관명_re'] = shop_edu_mapping_df['수요기관명'].str.replace('교육청','').str.replace('교육지원청','')
filter_edu1 = list(shop_edu_mapping_df[shop_edu_mapping_df['수요기관명_re'].str.split().str[0].notnull()]['수요기관명_re'].str.split().str[0].unique())
filter_edu2 = list(shop_edu_mapping_df[shop_edu_mapping_df['수요기관명_re'].str.split().str[1].notnull()]['수요기관명_re'].str.split().str[1].unique())
filter_edu3 = list(shop_edu_mapping_df[shop_edu_mapping_df['수요기관명_re'].str.split().str[2].notnull()]['수요기관명_re'].str.split().str[2].unique())

In [390]:
# 전체 키워드 데이터셋
edu_mapping_keywd_all = list(chain(*shop_edu_mapping_df['납품요구건명_re'].str.split(' '))) ## 전체 split 키워드
edu_mapping_keywd_df = pd.DataFrame(Counter(edu_mapping_keywd_all).items(),columns=['키워드','빈도수'])
edu_mapping_keywd_df = edu_mapping_keywd_df[edu_mapping_keywd_df['키워드'].apply(len)!=1].reset_index(drop=True)
edu_mapping_keywd_df.sort_values('빈도수',ascending=False)

,키워드,빈도수
2,인조잔디,130
1,운동장,84
15,구입,77
7,구매,64
55,관급자재인조잔디,63
...,...,...
164,토목공사,1
162,화산,1
161,목포부주,1
160,인조잔디구매,1


In [391]:
# 빅쿼리에서 불러오기
edu_budget_df = get_dataframe_from_bigquery('edu','edu_budget',bigquerykey_path)
edu_budget_df.sort_values('시도').reset_index(drop=True)

,시도,시군구,구분,과업명,금액,면적,예산집행
0,강원특별자치도,화천교육지원청,공립,상서중학교 운동장 보수,"89,533,000","4,664",학교
1,강원특별자치도,인제교육지원청,공립,상남중학교 개방형 운동장 조성,"725,302,000","6,507",교육청
2,강원특별자치도,강원특별자치도교육청,공립,강릉제일고등학교 축구장 인조잔디 교체,"1,020,560,000","8,576",교육청
3,강원특별자치도,강릉교육지원청,공립,관동중학교 인조잔디 운동장 조성,"806,400,000","4,082",교육청
4,강원특별자치도,강원특별자치도교육청,사립,동해광희고등학교 인조잔디운동장 보수,"51,700,000",None,None
...,...,...,...,...,...,...,...
189,충청남도,보령교육지원청,공립,성주초등학교 운동장 교체,"70,980,000",None,None
190,충청남도,당진교육지원청,공립,면천중학교 운동장 교체,"297,561,000",None,None
191,충청북도,충청북도교육청,공립,충북산업과학고등학교 다목적구장 조성,"769,799,000","5,540",교육청
192,충청북도,청주교육지원청,공립,서경중학교 운동장 보수 및 바닥교체,"602,871,000","3,500",교육청


In [392]:
# 전처리
edu_budget_df['과업명_re'] = edu_budget_df['과업명'].apply(clean_text)
edu_budget_df['과업명_re'] = edu_budget_df['과업명_re'].str.replace('초등학교','').str.replace('중학교','').str.replace('고등학교','')
edu_budget_df['과업명_re'] = edu_budget_df['과업명_re'].str.strip()

In [394]:
# 종합쇼핑몰 형식과 통일
edu_budget_df['시군구_re'] = np.where(edu_budget_df['시군구'].str.contains('교육지원청'),
                                   edu_budget_df['시도'] + edu_budget_df['시군구'].str.replace('교육지원청',''), edu_budget_df['시군구']) 

In [396]:
# 교육청, 교육지원청, 학교명 추출
filter_edu1_temp = pd.concat([edu_budget_df[edu_budget_df['시군구_re'].isin(filter_edu1)],
                              edu_budget_df[edu_budget_df['과업명'].str.contains(('|').join(filter_edu1))],
                              edu_budget_df[edu_budget_df['과업명_re'].str.contains(('|').join(filter_edu1))]],axis=0)
# edu_budget_df[edu_budget_df['시군구_re'].str.contains(('|').join(filter_edu1))].shape

filter_edu2_temp = pd.concat([edu_budget_df[edu_budget_df['시군구_re'].isin(filter_edu2)],
                              edu_budget_df[edu_budget_df['시군구_re'].str.contains(('|').join(filter_edu2))],
                              edu_budget_df[edu_budget_df['과업명'].str.contains(('|').join(filter_edu2))],
                              edu_budget_df[edu_budget_df['과업명_re'].str.contains(('|').join(filter_edu2))]],axis=0)

filter_edu3_temp = pd.concat([edu_budget_df[edu_budget_df['시군구_re'].isin(filter_edu3)],
                              edu_budget_df[edu_budget_df['시군구_re'].str.contains(('|').join(filter_edu3))],
                              edu_budget_df[edu_budget_df['과업명'].str.contains(('|').join(filter_edu3))],
                              edu_budget_df[edu_budget_df['과업명_re'].str.contains(('|').join(filter_edu3))]],axis=0)

filtered_edu_budget_df = pd.concat([filter_edu1_temp, filter_edu2_temp, filter_edu3_temp],axis=0).drop_duplicates().reset_index(drop=True)

In [406]:
# 종합쇼핑몰 학교 관련 키워드 빈도수
edu_filter_keywd = ('|').join(edu_mapping_keywd_df[edu_mapping_keywd_df['빈도수'] <= 3]['키워드'].unique())
edu_mapping_keywd_df[edu_mapping_keywd_df['빈도수'] <= 3].sort_values('빈도수',ascending=False) ## 빈도수 평균 2.99

,키워드,빈도수
43,다목적,3
25,운동장환경개선,3
3,교체,3
117,따른,3
47,물품,3
...,...,...
162,화산,1
161,목포부주,1
160,인조잔디구매,1
159,동북,1


In [420]:
# 교육청 제외 데이터
filtered_edu_budget_df_fin = filtered_edu_budget_df[filtered_edu_budget_df['과업명'].str.contains(edu_filter_keywd)].drop_duplicates().reset_index(drop=True)
filtered_edu_budget_df_fin

,시도,시군구,구분,과업명,금액,면적,예산집행,과업명_re,시군구_re
0,충청남도,보령교육지원청,공립,성주초등학교 운동장 교체,"70,980,000",None,None,성주 운동장 교체,충청남도보령
1,충청남도,보령교육지원청,공립,청파초등학교 운동장 교체,"20,976,000",None,None,청파 운동장 교체,충청남도보령
2,경기도,김포교육지원청,공립,신양중학교 친환경 운동장 조성 사업,None,None,None,신양 친환경 운동장 조성 사업,경기도김포
3,경기도,수원교육지원청,공립,수성중학교 친환경 운동장 조성 사업,None,None,None,수성 친환경 운동장 조성 사업,경기도수원
4,경기도,연천교육지원청,공립,백학중학교 친환경 운동장 조성 사업,None,None,None,백학 친환경 운동장 조성 사업,경기도연천
5,경기도,용인교육지원청,공립,어정초등학교 친환경 운동장 조성 사업,None,None,None,어정 친환경 운동장 조성 사업,경기도용인
6,경기도,용인교육지원청,공립,서원중학교 친환경 운동장 조성 사업,None,None,None,서원 친환경 운동장 조성 사업,경기도용인
7,경기도,파주교육지원청,공립,와석초등학교 친환경 운동장 조성 사업,None,None,None,와석 친환경 운동장 조성 사업,경기도파주
8,경기도,포천교육지원청,공립,영북초등학교 친환경 운동장 조성 사업,None,None,None,영북 친환경 운동장 조성 사업,경기도포천
9,서울특별시,동부교육지원청,공립,상봉중학교(운동장 환경 개선),"680,000,000","4,703",학교,상봉운동장 환경 개선,서울특별시동부


In [418]:
edu_budget_df[~edu_budget_df['과업명'].isin(filtered_edu_budget_df_fin['과업명'])].reset_index(drop=True)

,시도,시군구,구분,과업명,금액,면적,예산집행,과업명_re,시군구_re
0,전북특별자치도,전북특별자치도교육청,None,완주 스포츠클럽 실외야구장,"1,689,820,000",None,None,완주 스포츠클럽 실외야구장,전북특별자치도교육청
1,경기도,경기도교육청,None,친환경 운동장 조성사업비 지원 (30교),"15,000,000,000",None,학교,친환경 운동장 조성사업비 지원 교,경기도교육청
2,전라남도,전라남도교육청,None,운동장 개보수(고2),"1,652,038,000",None,학교,운동장 개보수고,전라남도교육청
3,울산광역시,울산광역시교육청,None,친환경운동장조성 다목적구장 중학교 1교,"14,000,000",None,학교,친환경운동장조성 다목적구장 교,울산광역시교육청
4,울산광역시,울산광역시교육청,None,친환경운동장조성 다목적구장 고등학교 6교,"465,600,000",None,학교,친환경운동장조성 다목적구장 교,울산광역시교육청
...,...,...,...,...,...,...,...,...,...
138,경상남도,의령교육지원청,사립,정곡중학교 체육관 신축(계속비),"38,000,000",None,학교,정곡 체육관 신축계속비,경상남도의령
139,대구광역시,대구광역시교육청,사립,대륜고등학교 축구장 인조잔디 재조성,"578,358,000","7,447",학교,대륜 축구장 인조잔디 재조성,대구광역시교육청
140,울산광역시,울산광역시교육청,사립,친환경운동장조성 다목적구장 중학교 (사립) 1교,"169,141,500",None,학교,친환경운동장조성 다목적구장 사립 교,울산광역시교육청
141,전북특별자치도,전북특별자치도교육청,사립,익산고등학교 인조잔디 조성,"1,050,001,000","7,417",교육청,익산 인조잔디 조성,전북특별자치도교육청
